# How to use MCDM methods
In this example, we will see how MCDM methods available in DESDEO can be utilized to solve a multiobjective optimization problem. MCDM methods are implemented in a functional way, which means their individual components must be first combined. This might feel unnecessary at first, but in practice, it means we are able to combine components of multiple methods to create new methods that can best suit our needs.

Before going into MCDM methods, we will first see examples of how scalarization is done in DESDEO, which is an important aspect of many MCDM methods. Throughout the this example, we will be solving the DTLZ2 problem with five variables and three objective functions. We choose this problem because of its simplicity and known shape of its Pareto front.

## Scalarization in DESDEO
Scalarization is the transformation of a multiobjective optimization problem in
to a single-objective optimization one. In DESDEO, this happens by adding a
`ScalarizationFunction` to an instance of the `Problem` class. However, because
many scalarization functions require information on either the ideal or nadir
point, or both, of the problem being solved, we first need to compute these before proceeding with scalarization.

### Computing the ideal point and (approximating) the nadir point
To compute the ideal point, and approximate the nadir point, of the DTLZ2
problem, we can utilize the payoff table method
(`desdeo.tools.payoff_table_method`). We begin by creating the DTLZ2 problem and
passing it ot the payoff table method:

In [8]:
from desdeo.problem.testproblems import dtlz2
from desdeo.tools import payoff_table_method

problem = dtlz2(5, 3)

ideal, nadir = payoff_table_method(problem)

While the computed `ideal` point will have its components all close to zero, as
expected, the nadir point will likely be a relatively bad approximation of the
true nadir point. We know that because the Pareto optimal front of the
3-dimensional variant of the `dtlz2` problem is the positive octant of a sphere,
then its nadir point should have the components $\left[1.0, 1.0, 1.0\right]$.
However, it is highly unlikely that the payoff table method will be able to
produce this point. This is because of the inherent limitations of the method.

In practice, the nadir point can be either inquired from a decision maker (domain
expert), or it can be approximated by solving
the problem utilizing an evolutionary method, such as NSGA3, and then reading
the nadir point values of the approximated Pareto front. However, while bad,
the payoff table method works out-of-the-box with almost any problem, and it 
gives at least a first workable approximation fo the nadir point. The ideal point
computed by the payoff table method is often accurate enough, however.

<div class="admonition note">
<p class="admonition-title">Note</p>
<p>The payoff table method (`desdeo.tools.payoff_table_method`) is a very rough
way to approximate the nadir point of a problem. It is better to use an
evolutionary method instead to approximate the problem's Pareto front, and then
read an approximated value for the nadir point from the front.
</p>
</div>

Regardless the source of the information on the nadir (and ideal) point values,
we can update the problem with this information

In [9]:
ideal = {"f_1": 0.0, "f_2": 0.0, "f_3": 0.0}
nadir = {"f_1": 1.0, "f_2": 1.0, "f_3": 1.0}

problem = problem.update_ideal_and_nadir(ideal, nadir)

It is important to notice how the utilized the method `update_ideal_and_nadir`
and then used its output to redefine the variable `problem`. In DESDEO, instance
of the `Problem` class are, in principle, __immutable__. This means that anytime
we wish to change an already instantiate `Problem` object, we essentially have
to create a new one. `Problem`s are chosen to be immutable in DESDEO to avoid
involuntarily changing the original problem, which can easily happen when
solving the problem with multiple methods.

<div class="admonition note">
<p class="admonition-title">Note</p>
<p>Instances of the <tt>Problem</tt> class in DESDEO are immutable. When making
changes to an existing `Problem` objects, methods and functions will return a
new instance of the original <tt>Problem</tt> with the applied changes.</p>
</div>

### Scalarizing a problem
Once we have the ideal and nadir point values available, we can scalarize our problem. We will use the achievement scalarizing function, or ASF. To utilize the ASF, we will need a reference point.
Let us use the values $\left[0.8, 0.4, 0.6\right]$ for the reference point, as an example. Because the DTLZ2 problem is differentiable, we can use the differentiable variant of the ASF (`desdeo.tools.add_asf_diff`) 

In [11]:
from desdeo.tools import add_asf_diff

reference_point = {"f_1": 0.8, "f_2": 0.4, "f_3": 0.6}

problem_w_asf = add_asf_diff(problem, symbol="asf", reference_point=reference_point)

When adding the ASF, we had to supply also a `symbol` in addition to the
`reference_point`. As discussed in the previous example, the `symbol` is
utilized to identify various components of a `Problem`. In this case, the
`symbol="asf"` is used to refer to the ASF, which was added to the problem.

We also mentioned that we can use the differentiable variant of the ASF, because
our problem (DTLZ2) is differentiable. This is important because we want to keep
the differentiability of our problem even after scalarization. This allows us to
use, e.g., gradient-based optimizers when solving the scalarized problem. In
turn, this allows us to get accurate and optimal solutions. If our problem was
not differentiable, then we could have used the non-differentiable variant of
ASF defined in `desdeo.tools.add_asf_nondiff`. This would be practical if we
knew we had to solve our problem using heuristics-based methods, which are
impartial to the differentiability of our problem. In fact, it could be even
detrimental to utilize the differentiable variant when solving the scalarized
problem with an evolutionary method, since the differentiable variant introduced
many constraints to the problem, which evolutionary methods are not very adept
to handle. The lesson here is, that we should be knowledgeable enough about our
problem to manipulate it in the best possible way.